## Problem 2: Fisher LDA

In [1]:
import numpy as np

### Step 1: Define the data points and their classes

In [2]:
X = np.array([
    [2.5, 1.0], # Class 0
    [2.0, 2.15], # Class 0
    [4.0, 2.9], # Class 1
    [3.6, 4.0] # Class 1
])

y = np.array([
    0,
    0,
    1,
    1
])

### Step 2: Compute class means

In [3]:
X_class_0 = X[y == 0]
X_class_1 = X[y == 1]

mu_1 = np.mean(X_class_0, axis=0)
mu_2 = np.mean(X_class_1, axis=0)

print("Mean of class 0 (mu_1):", mu_1)
print("Mean of class 1 (mu_2):", mu_2)

Mean of class 0 (mu_1): [2.25  1.575]
Mean of class 1 (mu_2): [3.8  3.45]


### Step 3: Compute the scatter matrices

In [19]:
# Center the data by subtracting the class mean
Xc_class_0 = X_class_0 - mu_1
Xc_class_1 = X_class_1 - mu_2

print("Centered data for class 0 (Xc_class_0):\n", Xc_class_0)
print("Centered data for class 1 (Xc_class_1):\n", Xc_class_1)

# Difference in means
mu_diff = (mu_2 - mu_1).reshape(-1, 1)
print("Difference between class means (mu_2 - mu_1):\n", mu_diff)

# Between-class scatter matrix (SB)
SB = np.dot(mu_diff, mu_diff.T)
print("Between-class scatter matrix (SB):\n", SB)

# Within-class scatter matrix
SC_1 = np.dot(Xc_class_0.T, Xc_class_0)
SC_2 = np.dot(Xc_class_1.T, Xc_class_1)

print("Within-class scatter matrix for class 0 (SC1):\n", SC_1)
print("Within-class scatter matrix for class 1 (SC2):\n", SC_2)

# Combine into total within-class scatter matrix
SW = SC_1 + SC_2

print("Total within-class scatter matrix (SW):\n", SW)

Centered data for class 0 (Xc_class_0):
 [[ 0.25  -0.575]
 [-0.25   0.575]]
Centered data for class 1 (Xc_class_1):
 [[ 0.2  -0.55]
 [-0.2   0.55]]
Difference between class means (mu_2 - mu_1):
 [[1.55 ]
 [1.875]]
Between-class scatter matrix (SB):
 [[2.4025   2.90625 ]
 [2.90625  3.515625]]
Within-class scatter matrix for class 0 (SC1):
 [[ 0.125   -0.2875 ]
 [-0.2875   0.66125]]
Within-class scatter matrix for class 1 (SC2):
 [[ 0.08  -0.22 ]
 [-0.22   0.605]]
Total within-class scatter matrix (SW):
 [[ 0.205   -0.5075 ]
 [-0.5075   1.26625]]


### Step 4: Compute the product of the matrices 

In [28]:
# Get the inverse
SW_inv_numpy = np.linalg.inv(SW)
print("Numpy inverse of SW (SW^-1):\n", SW_inv_numpy)

# Compute the product of SW^-1 and SB
SW_inv_SB = np.dot(SW_inv_numpy, SB)
print("Product of SW^-1 and SB (SW^-1 * SB):\n", SW_inv_SB)

Numpy inverse of SW (SW^-1):
 [[625.30864198 250.61728395]
 [250.61728395 101.2345679 ]]
Product of SW^-1 and SB (SW^-1 * SB):
 [[2230.66049383 2698.37962963]
 [ 896.32098765 1084.25925926]]


### Step 5: Find the optimal vector (w) 

In [32]:
# Compute the eigenvalues and eigenvectors of SW^-1 * SB
eigenvalues, eigenvectors = np.linalg.eig(SW_inv_SB)

# Display the eigenvalues
print("Eigenvalues of SW^-1 * SB:\n", eigenvalues)

# Optionally display the eigenvectors
print("Eigenvectors of SW^-1 * SB:\n", eigenvectors)

# Find the index of the largest eigenvalue
max_eigenvalue_index = np.argmax(eigenvalues)

# Get the largest eigenvalue and corresponding eigenvector
largest_eigenvalue = eigenvalues[max_eigenvalue_index]
largest_eigenvector = eigenvectors[:, max_eigenvalue_index]

# Display the largest eigenvalue and eigenvector
print("Largest Eigenvalue:", round(largest_eigenvalue, 2))
print("Corresponding Eigenvector:\n", largest_eigenvector)

Eigenvalues of SW^-1 * SB:
 [ 3.31491975e+03 -2.75876494e-14]
Eigenvectors of SW^-1 * SB:
 [[ 0.92789365 -0.77074232]
 [ 0.37284497  0.63714698]]
Largest Eigenvalue: 3314.92
Corresponding Eigenvector:
 [0.92789365 0.37284497]


### Step 6: Find the projections

In [39]:
w = largest_eigenvector  # The discriminant vector (eigenvector corresponding to the largest eigenvalue)
new_point = np.array([3.8, 
                      5])  # New point to classify

# Step 1: Project class means onto w
mu_w1 = np.dot(w, mu_1)  # Projection of class 1 mean onto w
mu_w2 = np.dot(w, mu_2)  # Projection of class 2 mean onto w
print("Projection of class 1 mean (mu_w1) =", round(mu_w1, 3), "* w")
print("Projection of class 2 mean (mu_w2) =", round(mu_w2, 3), "* w")

# Step 2: Compute the separation point (sep)
sep = (mu_w1 + mu_w2) / 2
print("Separation point (sep) = ", round(sep, 3), " * w")

# Step 3: Project the new point onto w
proj_new_point = np.dot(w, new_point)
print("Projection of new point onto w = ", round(proj_new_point, 3), " * w")

# Step 4: Classify the new point
if proj_new_point > sep:
    classification = "Class 2"
else:
    classification = "Class 1"

print("Classification of the new point:", classification)

Projection of class 1 mean (mu_w1) = 2.675 * w
Projection of class 2 mean (mu_w2) = 4.812 * w
Separation point (sep) =  3.744  * w
Projection of new point onto w =  5.39  * w
Classification of the new point: Class 2


# This is a redo for a universal case

In [40]:
import numpy as np

# Function to compute Fisher's LDA and classify a new point
def fisher_lda(X, y, new_point):
    # Separate data by class
    classes = np.unique(y)  # Find unique class labels
    class_means = {}
    scatter_within = np.zeros((X.shape[1], X.shape[1]))  # Initialize SW

    for c in classes:
        X_class = X[y == c]
        mu_c = np.mean(X_class, axis=0)
        class_means[c] = mu_c
        # Centered data for each class
        Xc_class = X_class - mu_c
        scatter_within += np.dot(Xc_class.T, Xc_class)

    print("Class Means:\n", class_means)
    print("Within-Class Scatter Matrix (SW):\n", scatter_within)

    # Between-class scatter matrix
    mu_diff = class_means[classes[1]] - class_means[classes[0]]
    mu_diff = mu_diff.reshape(-1, 1)  # Reshape to column vector
    scatter_between = np.dot(mu_diff, mu_diff.T)
    print("Between-Class Scatter Matrix (SB):\n", scatter_between)

    # Compute SW^-1 * SB
    SW_inv = np.linalg.inv(scatter_within)
    SW_inv_SB = np.dot(SW_inv, scatter_between)
    print("Product of SW^-1 and SB:\n", SW_inv_SB)

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(SW_inv_SB)
    max_eigenvalue_index = np.argmax(eigenvalues)
    w = eigenvectors[:, max_eigenvalue_index]  # Discriminant vector
    print("Largest Eigenvalue:", round(eigenvalues[max_eigenvalue_index], 3))
    print("Corresponding Eigenvector (w):\n", w)

    # Project means onto w
    mu_w1 = np.dot(w, class_means[classes[0]])
    mu_w2 = np.dot(w, class_means[classes[1]])
    print("Projection of class 1 mean (mu_w1) =", round(mu_w1, 3), "* w")
    print("Projection of class 2 mean (mu_w2) =", round(mu_w2, 3), "* w")

    # Separation point
    sep = (mu_w1 + mu_w2) / 2
    print("Separation point (sep) =", round(sep, 3), "* w")

    # Classify the new point
    proj_new_point = np.dot(w, new_point)
    print("Projection of new point onto w =", round(proj_new_point, 3), "* w")

    if proj_new_point > sep:
        classification = classes[1]
    else:
        classification = classes[0]

    print("Classification of the new point:", classification)
    return classification


In [41]:
# Example usage
X = np.array([
    [2.5, 1.0],  # Class 0
    [2.0, 2.15],  # Class 0
    [4.0, 2.9],   # Class 1
    [3.6, 4.0]    # Class 1
])

y = np.array([0, 0, 1, 1])  # Class labels
new_point = np.array([3.8, 5])  # New point to classify

# Call the Fisher LDA function
fisher_lda(X, y, new_point)


Class Means:
 {np.int64(0): array([2.25 , 1.575]), np.int64(1): array([3.8 , 3.45])}
Within-Class Scatter Matrix (SW):
 [[ 0.205   -0.5075 ]
 [-0.5075   1.26625]]
Between-Class Scatter Matrix (SB):
 [[2.4025   2.90625 ]
 [2.90625  3.515625]]
Product of SW^-1 and SB:
 [[2230.66049383 2698.37962963]
 [ 896.32098765 1084.25925926]]
Largest Eigenvalue: 3314.92
Corresponding Eigenvector (w):
 [0.92789365 0.37284497]
Projection of class 1 mean (mu_w1) = 2.675 * w
Projection of class 2 mean (mu_w2) = 4.812 * w
Separation point (sep) = 3.744 * w
Projection of new point onto w = 5.39 * w
Classification of the new point: 1


np.int64(1)